In [1]:
import pandas as pd
import numpy as np

In [15]:
img_to_ncbi = pd.read_csv("img_accession_to_ncbi_accession.xls", sep="\t")
img_to_ncbi.head(3)

,taxon_oid,Domain,Sequencing Status,Study Name,Genome Name / Sample Name,Sequencing Center,IMG Genome ID,Genus,IMG Submission ID,GOLD Analysis Project ID,...,NCBI Taxon ID,RefSeq Project ID,Strain,NCBI Assembly Accession,NCBI Bioproject Accession,NCBI Biosample Accession,NCBI GenBank ID,Genome Size * assembled,Gene Count * assembled,Unnamed: 20
0,2872623163,Bacteria,Finished,Genome sequencing of M. tuberculosis strains i...,Mycobacterium tuberculosis TBV4766,Universidad Peruana Cayetano Heredia,2872623163,Mycobacterium,232653,Ga0259373,...,1773,0,NaN,GCA_002448175.1,PRJNA407834,SAMN07664888,CP023638,4411310,4228,NaN
1,2619619571,Bacteria,Permanent Draft,Broad institute TB Antibiotic Resistance Catalog,Mycobacterium tuberculosis M1292,Broad Institute,2619619571,Mycobacterium,65254,Ga0045167,...,1448650,0,M1292,GCA_000654995.1,PRJNA235661,SAMN02598996,JKOQ00000000,4417793,4156,NaN
2,2565956582,Bacteria,Finished,Mycobacterium tuberculosis BT2,Mycobacterium tuberculosis BT2,Chinese National Human Genome Center at Shangh...,2565956582,Mycobacterium,36508,Ga0032248,...,1010835,0,BT2,GCA_000572155.1,PRJNA66063,SAMN03081430,CP002882,4401899,4197,NaN


In [27]:
x = pd.read_csv("genome_metadata", sep="\t")
x["genus"] = x.genome_name.apply(lambda row: row.split()[0].replace('"', "").replace("'", ""))
single_contigs = x[x.genus == "Mycobacterium"][["genome_id","genus", "contigs", "assembly_accession"]] #.genome_id.drop_duplicates().to_csv("all_mycobacterium_gids.txt", sep="\t", header=False, index=False)
single_contigs = single_contigs[single_contigs.contigs <= 1].dropna().drop_duplicates()
single_contigs.dropna(subset=["assembly_accession"]).to_csv("single_contig_mycobacteriums.csv")

In [45]:
x.columns

Index(['genome_id', 'genome_name', 'organism_name', 'taxon_id',
       'genome_status', 'strain', 'serovar', 'biovar', 'pathovar', 'mlst',
       'other_typing', 'culture_collection', 'type_strain', 'completion_date',
       'publication', 'bioproject_accession', 'biosample_accession',
       'assembly_accession', 'genbank_accessions', 'refseq_accessions',
       'sequencing_centers', 'sequencing_status', 'sequencing_platform',
       'sequencing_depth', 'assembly_method', 'chromosomes', 'plasmids',
       'contigs', 'sequences', 'genome_length', 'gc_content', 'patric_cds',
       'brc1_cds', 'refseq_cds', 'isolation_site', 'isolation_source',
       'isolation_comments', 'collection_date', 'isolation_country',
       'geographic_location', 'latitude', 'longitude', 'altitude', 'depth',
       'other_environmental', 'host_name', 'host_gender', 'host_age',
       'host_health', 'body_sample_site', 'body_sample_subsite',
       'other_clinical', 'antimicrobial_resistance',
       'antimic

In [38]:
img_to_ncbi["IMG Genome ID "] = img_to_ncbi["IMG Genome ID "].astype(str)
img_to_ncbi_dict = img_to_ncbi.set_index("IMG Genome ID ")["NCBI Assembly Accession"].to_dict()

In [42]:
img_bgcs = pd.read_csv("first_5000_Mycobacterium_BGCs_IMG_ABC.xls", sep="\t")
img_bgcs = img_bgcs[img_bgcs["Genome ID"].isin(img_to_ncbi_dict.keys())]
img_bgcs["assembly"] = img_bgcs["Genome ID"].apply(lambda row: img_to_ncbi_dict[row])
img_bgcs = img_bgcs[img_bgcs.assembly.isin(single_contigs.assembly_accession.unique())]
img_bgcs

,Cluster ID,Domain,Status,Genome ID,Genome Name,Scaffold ID,Start Coord,End Coord,Type,Method,Rules,Unnamed: 11,assembly
0,2506842941.Mycsm_Contig77.region1,Bacteria,Finished,2506783060,Mycobacterium sp. JS623,2506842941,196376,252524,"T1PKS,NRPS",antiSMASH v5.0,cds(PKS_AT and (PKS_KS or ene_KS or mod_KS or ...,NaN,GCA_000328565.1
1,2506842941.Mycsm_Contig77.region10,Bacteria,Finished,2506783060,Mycobacterium sp. JS623,2506842941,4294384,4335520,T3PKS,antiSMASH v5.0,(Chal_sti_synt_C or Chal_sti_synt_N),NaN,GCA_000328565.1
2,2506842941.Mycsm_Contig77.region11,Bacteria,Finished,2506783060,Mycobacterium sp. JS623,2506842941,4848846,4869796,terpene,antiSMASH v5.0,(Terpene_synth or Terpene_synth_C or phytoene_...,NaN,GCA_000328565.1
3,2506842941.Mycsm_Contig77.region12,Bacteria,Finished,2506783060,Mycobacterium sp. JS623,2506842941,5452888,5504534,"NRPS,T1PKS",antiSMASH v5.0,(cds(Condensation and (AMP-binding or A-OX)) o...,NaN,GCA_000328565.1
4,2506842941.Mycsm_Contig77.region13,Bacteria,Finished,2506783060,Mycobacterium sp. JS623,2506842941,5598281,5609078,bacteriocin,antiSMASH v5.0,(strepbact or Antimicrobial14 or Bacteriocin_I...,NaN,GCA_000328565.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2676,2565970367.CP002882.region5,Bacteria,Finished,2565956582,Mycobacterium tuberculosis BT2,2565970367,1529318,1569866,T3PKS,antiSMASH v5.0,(Chal_sti_synt_C or Chal_sti_synt_N),NaN,GCA_000572155.1
2677,2565970367.CP002882.region6,Bacteria,Finished,2565956582,Mycobacterium tuberculosis BT2,2565970367,1709279,1753831,T1PKS,antiSMASH v5.0,cds(PKS_AT and (PKS_KS or ene_KS or mod_KS or ...,NaN,GCA_000572155.1
2678,2565970367.CP002882.region7,Bacteria,Finished,2565956582,Mycobacterium tuberculosis BT2,2565970367,1851139,1909266,"T3PKS,T1PKS",antiSMASH v5.0,"(Chal_sti_synt_C or Chal_sti_synt_N),cds(PKS_A...",NaN,GCA_000572155.1
2679,2565970367.CP002882.region8,Bacteria,Finished,2565956582,Mycobacterium tuberculosis BT2,2565970367,2260692,2312931,T1PKS,antiSMASH v5.0,cds(PKS_AT and (PKS_KS or ene_KS or mod_KS or ...,NaN,GCA_000572155.1


In [82]:
smash = pd.read_csv("antismash_db_results.csv", sep="\t")
smash[smash["NCBI accession"] == "NC_010612.1"] 
counts = smash["NCBI accession"].value_counts()
above_10_index = counts[counts > 10].index
smash = smash[smash["NCBI accession"].isin(above_10_index)]
smash["NCBI accession"] = smash["NCBI accession"].apply(lambda row: row.split(".")[0])
smash_dict = dict()
for accession, df in smash.groupby("NCBI accession"):
    smash_dict[accession] = df[["From", "To"]].to_numpy()

In [ ]:
smash_dict